ANN MODEL

In [32]:
import pandas as pd
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk#for text wrangling
import spacy#for text wrangling
import re
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim


In [33]:
df = pd.read_csv("mail_data.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [34]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [35]:
sum(df.Message.notna())

5572

In [36]:
sum(df.Category.notna())
#there is no null category as well as no null message so we dont need to handle that.

5572

In [37]:
X = df['Message']
Y = df['Category']

In [38]:
STOPWORDS = stopwords.words("english")

In [39]:
#before encoding our messages to vectors using for now TF-IDF Vectorizer we will do text pre processing
nlp = spacy.load("en_core_web_sm")
def text_preprocess(text):
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r'[^a-zA-Z\s!$]'," ",text)#remove anything that is not a space or a character
    text = re.sub(r'\s+'," ",text)#normalize extra word spaces to a single space
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if token.lemma_.lower() not in STOPWORDS and not token.is_space and not token.is_punct]
    return " ".join(tokens)

X = X.apply(text_preprocess)
    

In [40]:
X

0       go jurong point crazy available bugis n great ...
1                                   ok lar joking wif oni
2       free entry wkly comp win fa cup final tkts st ...
3                         dun say early hor c already say
4                     nah think go usf live around though
                              ...                        
5567    nd time try contact win pound prize claim easy...
5568                               b go esplanade fr home
5569                                 pity mood suggestion
5570    guy bitching act like would interested buy som...
5571                                       rofl true name
Name: Message, Length: 5572, dtype: object

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [50]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    stop_words='english',
    max_features = 5000
)
X_enc = vectorizer.fit_transform(X)

In [52]:
X_dense = X_enc.toarray()

In [62]:
le= LabelEncoder()
y_enc = le.fit_transform(Y)

In [65]:
X_train,X_test,y_train,y_test = train_test_split(X_dense,y_enc,random_state=42)

In [66]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Turn it into a dictionary for Keras
class_weight_dict = {i : class_weights[i] for i in range(len(class_weights))}

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_enc.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2,class_weight=class_weight_dict)


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8641 - loss: 0.6235 - val_accuracy: 0.9761 - val_loss: 0.1473
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9797 - loss: 0.1104 - val_accuracy: 0.9725 - val_loss: 0.0889
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9915 - loss: 0.0345 - val_accuracy: 0.9701 - val_loss: 0.0779
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9965 - loss: 0.0158 - val_accuracy: 0.9725 - val_loss: 0.0855
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9985 - loss: 0.0083 - val_accuracy: 0.9725 - val_loss: 0.0902
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0048 - val_accuracy: 0.9725 - val_loss: 0.0947
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9997 - loss: 0.0024 - val_accuracy: 0.9725 - val_loss: 0.1021
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.

In [70]:
# Predict on test data
y_pred = model.predict(X_test)  # Returns probabilities for each test sample

# Convert probabilities to classes
y_pred_classes = (y_pred > 0.5).astype(int)

# Evaluate the model's performance
loss, accuracy = model.evaluate(X_test, y_test)

# Optionally, print out the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_classes))


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.9784 - loss: 0.1179
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1207
           1       0.96      0.88      0.92       186

    accuracy                           0.98      1393
   macro avg       0.97      0.94      0.95      1393
weighted avg       0.98      0.98      0.98      1393



In [76]:
my_text = "Congrats! You’ve won a free iPhone. Call now!"
my_text = text_preprocess(my_text)

In [77]:
# Convert to the same format as the training data (using the same vectorizer)
input_vector = vectorizer.transform([my_text]).toarray()
# Predict the class (0 for ham, 1 for spam)
prediction = model.predict(input_vector)
predicted_class = (prediction > 0.5).astype(int)  # 0 = ham, 1 = spam
if predicted_class == 1:
    print("This message is SPAM!")
else:
    print("This message is HAM.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
This message is SPAM!


In [ ]:
my_text = "Congratulations for your . Call now!"
my_text = text_preprocess(my_text)